In [ ]:
import requests 
import os  
from openai import OpenAI 

# Set up API credentials and endpoints
openai_api_key=os.getenv("OPENAI_API_KEY")  # Get OpenAI API key from environment variables
api_url="https://api.studio.nebius.ai"  # Base URL for the Nebius AI Studio API
base_url = api_url+"/v1"  # v1 API endpoint

# Initialize the OpenAI client with custom base URL and API key
client = OpenAI(
    base_url=base_url,
    api_key=os.getenv('NB_STUDIO_API_KEY', 'ANY'),  # Use Nebius Studio API key or fallback to 'ANY'
)

def upload_file(file_name):
    # Upload the file to the API server
    with open(file_name, "rb") as file_data:
        files = {"file": (os.path.basename(file_name), file_data)}
        upload_response = requests.post(
            f"{api_url}/v0/models/upload",
            files=files,
            headers={"Authorization": f"Bearer {openai_api_key}"}
        )
    
    # Check if the upload was successful
    if upload_response.status_code != 200:
        print(f"Error uploading file: {upload_response.text}")
        return upload_response.json()
    
    # Get and return the uploaded file ID
    file_info = upload_response.json()
    file_id = file_info["id"]
    print(f"File uploaded successfully with ID: {file_id}")
    return file_id

def create_lora_from_file(name, file_id, base_model):    
    # Create the LoRA model using the uploaded file
    lora_creation_request = {
        "source": file_id,  # ID of the uploaded training data
        "base_model": base_model,  # Base model to adapt
        "name": name,  # Name for the new model
        "description": "description"  # Description of the model
    }
    
    # Send request to create the model
    model_response = requests.post(
        f"{api_url}/v0/models", 
        json=lora_creation_request,
        headers={"Content-Type": "application/json", "Authorization": f"Bearer {openai_api_key}"}
    ) 
    
    return(model_response.json())

def delete_lora(model):
    return requests.delete(f"{api_url}/v0/models/{model}",                         
      headers={"Content-Type": "application/json","Authorization": f"Bearer {os.getenv('NB_STUDIO_API_KEY')}"})  


def list_loras():
    return requests.get(f"{api_url}/v0/models",                         
      headers={"Content-Type": "application/json","Authorization": f"Bearer {os.getenv('NB_STUDIO_API_KEY')}"})  

def get_completion(model):
    completion = client.chat.completions.create(
        model=model,
        messages=[{"role":"user","content":'hi'}],  # Simple test prompt
    )
    return completion.choices[0].message.content


# Define path to the lora adapter archive file with adapter_config.json and adapter_model.safetensors
zip_file_name="/Users/sofrony/tmp/LORA.zip"
    # Upload the file and get the file ID
file_id = upload_file(zip_file_name)

# Create a new LoRA model using the file_id
create_lora_from_file("Sofrony-Test", file_id, "meta-llama/Meta-Llama-3.1-8B-Instruct")

File uploaded successfully with ID: file-878b7628-6ec5-4915-9ce9-240cda24bd81


{'name': 'meta-llama/Meta-Llama-3.1-8B-Instruct-LoRa:Sofrony-Test-hUUO',
 'base_model': 'meta-llama/Meta-Llama-3.1-8B-Instruct',
 'source': 'file-878b7628-6ec5-4915-9ce9-240cda24bd81',
 'description': 'description',
 'created_at': 1744635333,
 'status': 'validating'}

In [ ]:
last_lora = list_loras().json()[-1]  # list loras and get the last one
last_lora

{'type': 'text2text',
 'name': 'meta-llama/Meta-Llama-3.1-8B-Instruct-LoRa:Sofrony-Test-hUUO',
 'status': 'active',
 'status_reason': None,
 'checkpoint_id': None,
 'job_id': None,
 'file_id': 'file-878b7628-6ec5-4915-9ce9-240cda24bd81',
 'url': None,
 'created_at': 1744635333,
 'description': 'description',
 'vendor': 'meta',
 'tags': ['128K context', 'small', 'JSON mode', 'lora'],
 'use_cases': ['lora'],
 'quality': 73,
 'context_window_k': 128,
 'size_b': 8.03}

In [11]:
get_completion(last_lora["name"])

'How can I assist you today?'

In [ ]:
delete_lora(last_lora["name"]).json()

{'name': 'meta-llama/Meta-Llama-3.1-8B-Instruct-LoRa:Sofrony-Test-hUUO',
 'base_model': 'meta-llama/Meta-Llama-3.1-8B-Instruct',
 'source': 'file-878b7628-6ec5-4915-9ce9-240cda24bd81',
 'description': 'description',
 'created_at': 1744635333,
 'status': 'deleted'}

In [ ]:
def create_lora_from_url(name, url):
    lora_creation_request = {
        "source": url,
        "base_model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
        "name": name,
        "description": "sofrony LoRA adapter"
    }
    response = requests.post(f"{api_url}/v0/models", 
        json=lora_creation_request,
        headers={"Content-Type": "application/json", "Authorization": f"Bearer {os.getenv('NB_STUDIO_API_KEY')}"}) 
    return response.json()

create_lora_from_url('sofrony-URL', 'huggingface.co/sofrony/Llama-3.1-8B-Instruct-sofrony')


{'name': 'meta-llama/Meta-Llama-3.1-8B-Instruct-LoRa:sofrony-URL-cdRh',
 'base_model': 'meta-llama/Meta-Llama-3.1-8B-Instruct',
 'source': 'sofrony/Llama-3.1-8B-Instruct-sofrony',
 'description': 'sofrony LoRA adapter',
 'created_at': 1744636074,
 'status': 'validating'}

In [2]:
get_completion("meta-llama/Meta-Llama-3.1-8B-Instruct-LoRa:sofrony-URL-cdRh")

'How can I assist you today?'

In [3]:
delete_lora('meta-llama/Meta-Llama-3.1-8B-Instruct-LoRa:sofrony-URL-cdRh').json()

{'name': 'meta-llama/Meta-Llama-3.1-8B-Instruct-LoRa:sofrony-URL-cdRh',
 'base_model': 'meta-llama/Meta-Llama-3.1-8B-Instruct',
 'source': 'sofrony/Llama-3.1-8B-Instruct-sofrony',
 'description': 'sofrony LoRA adapter',
 'created_at': 1744636074,
 'status': 'deleted'}